In [1]:
from meshparty import meshwork
import pandas as pd
import pcg_skel

In [12]:
from caveclient import CAVEclient
client = CAVEclient('minnie65_phase3_v1')

In [14]:
client.chunkedgraph.server_version

<Version('2.17.2')>

In [2]:
skdir = '/Users/caseysm/Work/Projects/pt_targets/skeletons'
root_id = 864691135273655569

nrn = meshwork.load_meshwork(
    f"{skdir}/{root_id}.h5"
)

In [3]:
pcg_skel.features.add_synapse_count(
    nrn,
)

In [9]:
nrn.save_meshwork("tests/data/test_neuron.h")

In [10]:
client.materialize.

NameError: name 'client' is not defined

In [4]:
nrn.anno.synapse_count.df.head()

,mesh_index,num_syn_out,net_size_out,num_syn_in,net_size_in,mesh_index_filt
0,0,0,0.0,0,0.0,0
1,1,0,0.0,0,0.0,1
2,2,2,37144.0,1,916.0,2
3,3,0,0.0,0,0.0,3
4,4,0,0.0,0,0.0,4


In [8]:
skel_df = pcg_skel.features.aggregate_property_to_skeleton(
    nrn,
    anno_table='synapse_count',
    agg_dict={
        'num_syn_out': 'sum',
        'num_syn_in': 'sum',
        'net_size_out': 'sum',
        'net_size_in': 'sum',
    },
)

In [13]:
nrn.anno['synapse_count'].df[['num_syn_out', 'num_syn_in', 'net_size_out', 'net_size_in']].sum()

num_syn_out           585.0
num_syn_in          11823.0
net_size_out      5157480.0
net_size_in     104518760.0
dtype: float64

In [32]:
set(nrn.anno.synapse_count._original_columns).difference(
    set([nrn.anno.synapse_count.index_column_original]),
) 

{'net_size_in', 'net_size_out', 'num_syn_in', 'num_syn_out'}

In [31]:
set(nrn.anno.synapse_count._original_columns).difference({'mesh_index'})

{'net_size_in', 'net_size_out', 'num_syn_in', 'num_syn_out'}

In [29]:
set([1,2,3]).difference({1})

{2, 3}

In [14]:
skel_df.sum()

num_syn_out           585.0
num_syn_in          11823.0
net_size_out      5157480.0
net_size_in     104518760.0
dtype: float64

In [ ]:
nrn.anno.synapse_count.df

In [6]:
syn_df = nrn.anno.synapse_count.df

In [8]:
syn_df['skel_index'] = nrn.anno.synapse_count.mesh_index.to_skel_index_padded

In [10]:
skel_syn_df = syn_df.groupby('skel_index').sum()[['num_syn_in', 'num_syn_out', 'net_size_in', 'net_size_out']]

In [13]:
skel_syn_df['mean_syn_in'] = (skel_syn_df['net_size_in'] / skel_syn_df['num_syn_in']).fillna(0)
skel_syn_df['mean_syn_out'] = (skel_syn_df['net_size_out'] / skel_syn_df['num_syn_out']).fillna(0)

In [26]:
import numpy as np

In [ ]:
nrn.skeleton.n_vertices

In [26]:
from caveclient import CAVEclient
client = CAVEclient('minnie65_phase3_v1')

In [ ]:
np.arange(0,10)

In [79]:
import numpy as np

In [57]:
syn_count_df = pd.DataFrame(
    index=nrn.mesh_indices
)
syn_out_df = nrn.anno.pre_syn.df.groupby('pre_pt_mesh_ind').agg(
    num_syn_out=pd.NamedAgg('id', 'count'),
    mean_size_out=pd.NamedAgg('size', 'mean'),
)

syn_in_df = nrn.anno.post_syn.df.groupby('post_pt_mesh_ind').agg(
    num_syn_in=pd.NamedAgg('id', 'count'),
    mean_size_in=pd.NamedAgg('size', 'mean'),
)

syn_count_df = syn_count_df.merge(
    syn_out_df,
    left_index=True,
    right_index=True,
    how='left',
).merge(
    syn_in_df,
    left_index=True,
    right_index=True,
    how='left',
).fillna(0)

syn_count_df['num_syn_out'] = syn_count_df['num_syn_out'].astype(int)
syn_count_df['num_syn_in'] = syn_count_df['num_syn_in'].astype(int)
syn_count_df = syn_count_df.reset_index()

nrn.anno.add_annotations(
    'syn_count',
    syn_count_df,
    index_column='index',
)

In [ ]:
nrn.skeleton.mesh_to_skel_map

In [89]:
syn_count_df = nrn.anno.syn_count.df
syn_count_df['skel_index'] = nrn.skeleton.mesh_to_skel_map


In [ ]:
nrn.anno.synapse_count.index_column_original

In [ ]:
client.materialize.tables.nucleus_detection_v0(pt_root_id=864691135464946757).query()

In [94]:
qry = client.materialize.tables.nucleus_detection_v0(pt_root_id=864691135464946757)

In [ ]:
qry.query(split_positions=True)

In [ ]:
client.materialize.query_table(
    "nucleus_detection_v0",
    **qry.filter_kwargs_mat
)

In [ ]:
qry.filter_kwargs_mat

In [72]:
with nrn.mask_context( nrn.anno.is_axon.mesh_mask):
    df = nrn.anno.syn_count.df

In [ ]:
df.num_syn_out.sum()

In [ ]:
df.num_syn_in.sum()